In [1]:
import pandas as pd
import plotly.express as px
pd.options.plotting.backend = "plotly"

In [2]:
df = pd.read_csv("trialdataformodel.csv", index_col=0)

df[["mating.order"]] = df[["mating.order"]].astype(str)

df.head()

,mating.ID,cohort,female.ID,female.group,female.age,female.wt,male.ID,male.qual,male.age,male.wt,...,female.aggression,male.w.gift,mating.success,gift.acceptance,signal,wrapping,rubbing,latency.gift,latency.cop,cop.duration
1,116x85,1,116,HL,12,0.1347,85,H,15,0.0670,...,N,1,1,0,0,0,1,NaN,87.0,390
2,116x86,1,116,HL,12,0.1347,86,L,15,0.0788,...,N,1,1,1,0,0,1,529.0,606.0,3869
3,122x75,1,122,LH,12,0.1818,75,L,15,0.0703,...,N,1,1,1,0,0,0,293.0,559.0,221
4,122x76,1,122,LH,12,0.1818,76,H,24,0.0621,...,Y,1,0,0,0,0,0,NaN,NaN,0
5,123x86,1,123,LH,13,0.1215,86,L,15,0.0788,...,N,0,1,1,1,1,1,2092.0,162.0,1252


In [3]:
groupdiscrete = df.groupby(["male.qual", "mating.order", "female.group"]).describe()[[("mating.success","count")]]


groupdiscrete = groupdiscrete.reset_index().droplevel(1, axis=1)

px.histogram(groupdiscrete, x="male.qual", y="mating.success", color="mating.order", barmode='group')

fig = px.bar(groupdiscrete,
             x="mating.order", y="mating.success", barmode="group",
             color="male.qual"
                              )

fig.show()

In [14]:
groupcontinous = df[["male.qual", "mating.order", "female.group","latency.gift", "latency.cop", "cop.duration"]].groupby(["male.qual", "mating.order", "female.group"]).mean()
groupcontinousstd = df[["male.qual", "mating.order", "female.group","latency.gift", "latency.cop", "cop.duration"]].groupby(["male.qual", "mating.order", "female.group"]).std()

groupcontinous = groupcontinous.reset_index()
groupcontinousstd = groupcontinousstd.reset_index()
groupcontinous[["mating.success"]] = groupdiscrete[["mating.success"]]
groupcontinous[["latency.gift.std"]] = groupcontinousstd[["latency.gift"]]
groupcontinous[["latency.cop.std"]] = groupcontinousstd[["latency.cop"]]
groupcontinous[["cop.duration.std"]] = groupcontinousstd[["cop.duration"]]
groupcontinous


,male.qual,mating.order,female.group,latency.gift,latency.cop,cop.duration,mating.success,latency.gift.std,latency.cop.std,cop.duration.std
0,H,1,HL,401.080000,473.807692,2075.300000,30.0,262.935974,279.198284,1542.514364
1,H,2,LH,548.611111,777.571429,1078.214286,28.0,714.581008,674.457014,1669.544346
2,L,1,LH,556.000000,595.320000,2177.250000,28.0,487.862406,370.531906,1641.372633
3,L,2,HL,525.250000,528.411765,1776.724138,29.0,581.882749,452.284487,2041.139981


In [17]:

fig = px.scatter_matrix(groupcontinous, dimensions=["latency.gift", "latency.cop", "cop.duration"], color="male.qual", symbol="mating.order", size="mating.success", )
fig.update_traces(diagonal_visible=False)
fig.show()
fig.write_html("matrix.html", include_plotlyjs="cdn")

fig = px.scatter(groupcontinous, x="latency.gift", y="cop.duration", color="male.qual", symbol="mating.order", size="mating.success", error_x = "latency.gift.std", error_y = "cop.duration.std")
fig.show()
fig.write_html("scattergiftduration.html", include_plotlyjs="cdn")
fig = px.scatter(groupcontinous, x="latency.gift", y="latency.cop", color="male.qual", symbol="mating.order", size="mating.success", error_x = "latency.gift.std", error_y = "latency.cop.std")
fig.show()
fig.write_html("scattergiftlatcop.html", include_plotlyjs="cdn")
fig = px.scatter(groupcontinous, x="latency.cop", y="cop.duration", color="male.qual", symbol="mating.order", size="mating.success", error_x = "latency.cop.std", error_y = "cop.duration.std")
fig.show()
fig.write_html("scatterlatcopduration.html", include_plotlyjs="cdn")




In [18]:
fig = px.violin(df, y="latency.gift", x="male.qual", color="mating.order", box=True, points="all",
          hover_data=df.columns)
fig.show()
fig.write_html("violinlatgift.html", include_plotlyjs= "cdn")
fig = px.violin(df, y="latency.cop", x="male.qual", color="mating.order", box=True, points="all",
          hover_data=df.columns)
fig.show()
fig.write_html("violinlatcop.html", include_plotlyjs= "cdn")
fig = px.violin(df, y="cop.duration", x="male.qual", color="mating.order", box=True, points="all",
          hover_data=df.columns)
fig.show()
fig.write_html("violincopduration.html", include_plotlyjs= "cdn")